In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from time import process_time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import psutil
import os
import re
import multiprocessing
import joblib
import gc
from tqdm.notebook import tqdm  # Progress bars for Jupyter

# Configuration options
VISUALIZE_RECTANGLES = False  # Set to True if you want to visualize rectangles
SAVE_INTERMEDIATE_MODELS = True  # Set to False to save only final models

# Flag to control whether to use multiple scales or only the maximum scale
use_multiple_scales = True  # Set to False to use only maximum scale, True for all scales

# Get available CPU cores and set appropriate parallelism
n_cores = multiprocessing.cpu_count()
n_jobs = max(1, n_cores - 1)  # Leave one core free for system processes
print(f"Using {n_jobs} of {n_cores} available CPU cores")

def monitor_memory():
    """Print current memory usage of the process"""
    process = psutil.Process()
    memory_mb = process.memory_info().rss / (1024 * 1024)
    print(f"Memory usage: {memory_mb:.2f} MB")

def MAPE(actual_values, predicted_values):
    """Calculate Mean Absolute Percentage Error with special handling for zeros"""
    # Vectorized implementation
    actual_flat = actual_values.flatten()
    pred_flat = predicted_values.flatten()
    
    # Create mask for non-zero actual values
    non_zero_mask = actual_flat != 0
    zero_mask = ~non_zero_mask
    
    # Calculate MAPE for non-zero elements
    mape_sum = 0
    count = len(actual_flat)
    
    if np.any(non_zero_mask):
        mape_sum += np.sum(np.abs((actual_flat[non_zero_mask] - pred_flat[non_zero_mask]) / actual_flat[non_zero_mask]))
    
    if np.any(zero_mask):
        mape_sum += np.sum(np.abs(actual_flat[zero_mask] - pred_flat[zero_mask]) / 100)
    
    return mape_sum / count

# Load spatial statistics to get universe boundaries for each dataset
print("Loading spatial statistics...")
spatial_stats = pd.read_csv('../spatial_statistics.csv')

# Directory containing the datasets
data_dir = '../large_files/resultsContains/'

# Parse bounding box information
def parse_bbox(bbox_str):
    # Extract coordinates from BOX string using regex
    pattern = r"BOX\(([-\d\.]+) ([-\d\.]+),([-\d\.]+) ([-\d\.]+)\)"
    match = re.search(pattern, bbox_str)
    if match:
        xmin = float(match.group(1))
        ymin = float(match.group(2))
        xmax = float(match.group(3))
        ymax = float(match.group(4))
        return xmin, ymin, xmax, ymax
    return -180, -90, 180, 90  # Default if parsing fails

# Extract universe boundaries for each dataset
universe_boundaries = {}
for _, row in spatial_stats.iterrows():
    table_name = row['Table Name']
    bbox = parse_bbox(row['Universe Limits (Bounding Box)'])
    universe_boundaries[table_name] = bbox

# Get list of all CSV files in the directory
print("Finding dataset files...")
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
print(f"Found {len(csv_files)} datasets to process")

# Define the scales of learning
scales = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]

# Create necessary directories
os.makedirs('../large_files/LearnedModels/contain/XGB', exist_ok=True)
os.makedirs('../large_files/LearnedModels/contain/XGB/visualizations', exist_ok=True)
os.makedirs('../large_files/LearnedModels/contain/XGB/results', exist_ok=True)

# Lists to store all results
all_results_list = []

# Process each dataset
for csv_file in tqdm(csv_files, desc="Processing datasets"):
    # Force garbage collection at the start of each dataset
    gc.collect()
    monitor_memory()
    
    # Extract dataset name (remove "_results.csv")
    dataset_name = csv_file.replace('_results.csv', '')
    
    print(f"\nProcessing dataset: {dataset_name}")
    
    # Get universe boundaries for this dataset
    if dataset_name in universe_boundaries:
        univ_xmin, univ_ymin, univ_xmax, univ_ymax = universe_boundaries[dataset_name]
    else:
        # Default values if dataset not found in spatial stats
        univ_xmin, univ_ymin, univ_xmax, univ_ymax = -180, -90, 180, 90
    
    Surface_univ = (univ_xmax - univ_xmin) * (univ_ymax - univ_ymin)
    print(f"Universe boundaries for {dataset_name}: ({univ_xmin}, {univ_ymin}, {univ_xmax}, {univ_ymax})")
    
    # Load dataset - only load required columns
    data_path = os.path.join(data_dir, csv_file)
    print(f"Loading data from {data_path}")
    data = pd.read_csv(data_path, usecols=['Query MBR', 'Count MBR'])
    
    # Extract query MBR column (needs parsing as it's in string format)
    def parse_mbr(mbr_str):
        coords = mbr_str.strip('"()').split(', ')
        return [float(coord) for coord in coords]
    
    # Extract columns - use list comprehension for better performance
    print("Parsing MBR coordinates...")
    Rectangles = np.array([parse_mbr(mbr) for mbr in data['Query MBR']])
    Y = data[['Count MBR']].values  # Using Count MBR as target
    
    # Free up memory
    del data
    gc.collect()
    
    # Calculate basic statistics
    max_count = float(np.max(Y))
    min_count = float(np.min(Y))
    mean_count = float(np.mean(Y))
    median_count = float(np.median(Y))
    total_samples = len(Y)

    # Display basic statistics for the dataset
    print(f"\nBasic statistics for {dataset_name} dataset:")
    print(f"Max count: {max_count}")
    print(f"Min count: {min_count}")
    print(f"Mean count: {mean_count:.2f}")
    print(f"Median count: {median_count:.2f}")
    print(f"Total samples: {total_samples}\n")

    # Calculate rectangles density - vectorized version
    print("Calculating rectangle densities...")
    width = Rectangles[:, 2] - Rectangles[:, 0]
    height = Rectangles[:, 3] - Rectangles[:, 1]
    rectanglesDensity = np.abs(width * height / Surface_univ).reshape(-1, 1)
    
    # Prepare the dataset
    # X = np.append(Rectangles, rectanglesDensity, axis=1)
    X = Rectangles
    
    # Split the data into 80% train and 20% test
    print("Splitting data into train and test sets...")
    X_train, X_test_all, y_train, y_test_all = train_test_split(X, Y, test_size=0.2, random_state=3)
    
    # Visualize the first 1000 rectangles (only if enabled)
    if len(Rectangles) > 0 and VISUALIZE_RECTANGLES:
        print("Visualizing rectangles sample...")
        plt.figure(figsize=(10, 8))
        ax = plt.subplot()
        
        # Only visualize a sample to save time
        sample_size = min(1000, len(Rectangles))
        for i in range(sample_size):
            x1, y1, x2, y2 = Rectangles[i]
            color_val = float(rectanglesDensity[i][0]) if hasattr(rectanglesDensity[i], '__len__') else float(rectanglesDensity[i])
            rectangle = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, 
                                        linewidth=1, edgecolor='b', facecolor='none', alpha=min(1.0, color_val*10))
            ax.add_patch(rectangle)
            
        plt.xlim(univ_xmin-20, univ_xmax+20)
        plt.ylim(univ_ymin-10, univ_ymax+10)
        plt.title(f"Sample rectangles from {dataset_name}")
        plt.savefig(f"../large_files/LearnedModels/contain/XGB/visualizations/{dataset_name}_rectangles.png", dpi=150)
        plt.close()  # Close to free memory instead of plt.show()
    
    # Adjust scales to the dataset size
    max_size = len(X_train)
    print(f"Training set size: {max_size}")

    if use_multiple_scales:
        # Use multiple scales as before
        adjusted_scales = [s for s in scales if s <= max_size]
        
        # Add intermediate 1 million increments for large datasets
        if max_size > 1000000:
            million_increments = list(range(2000000, max_size, 1000000))
            adjusted_scales.extend(million_increments)
            
        # Add the actual max size if it's not already in the list
        if max_size not in adjusted_scales:
            adjusted_scales.append(max_size)
            
        # Sort the scales to ensure they're in ascending order
        adjusted_scales.sort()
    else:
        # Use only the maximum scale
        adjusted_scales = [max_size]

    # List to store dataset-specific results
    dataset_results_list = []

    # Store best parameters from max scale training to reuse
    best_params = None
    
    # Process scales in reversed order (largest first)
    for sample_size in reversed(adjusted_scales):
        print(f"\nTraining with sample size: {sample_size}")
        monitor_memory()
        
        # Create training subset
        X_train_sample = X_train[:sample_size, :]
        y_train_sample = y_train[:sample_size]
        
        # XGBoost parameters - optimized for performance
        params_xgb = {
            "max_depth": [3, 5, 7],
            "learning_rate": [0.01, 0.1, 0.3],
            "n_estimators": [50, 100, 200]
        }
        
        # For very large datasets, use smaller parameter grid
        # if sample_size > 100000:
        #     params_xgb = {
        #         "max_depth": [5],
        #         "learning_rate": [0.1],
        #         "n_estimators": [100]
        #     }
            
        # Only do GridSearch for the max scale
        if sample_size == max_size or best_params is None:
            print("Performing grid search for optimal parameters...")
            # Using tree_method='hist' for faster training
            xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', 
                                           tree_method='hist', 
                                           random_state=3,
                                           n_jobs=n_jobs)
            xgb_cv = GridSearchCV(xgb_regressor, params_xgb, cv=3, n_jobs=1, verbose=1)  # Use n_jobs=1 as XGBoost uses parallel processing
            
            # Time the grid search
            t1_start = process_time()
            xgb_cv.fit(X_train_sample, y_train_sample.ravel())  # Use ravel for 1D array
            t1_stop = process_time()
            grid_search_time = t1_stop - t1_start
            
            # Store best parameters for reuse
            best_params = xgb_cv.best_params_
            print(f"Grid search complete in {grid_search_time:.2f}s")
            print(f"Best parameters: {best_params}")
        else:
            # Skip grid search for smaller scales, use params from max scale
            xgb_cv = None
            grid_search_time = 0
            print(f"Using best parameters from max scale: {best_params}")
        
        # Train the model with best parameters
        print("Training XGBoost model...")
        # Using tree_method='hist' for faster training and better memory efficiency
        xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror',
                                       tree_method='hist',
                                       random_state=3,
                                       n_jobs=n_jobs,
                                       early_stopping_rounds=10,
                                       **best_params)
        
        t2_start = process_time()
        xgb_regressor.fit(X_train_sample, y_train_sample.ravel(), 
                        eval_set=[(X_train_sample[-min(1000, len(X_train_sample)):], 
                                 y_train_sample[-min(1000, len(X_train_sample)):].ravel())],
                        verbose=False)
        t2_stop = process_time()
        training_time = t2_stop - t2_start
        
        # Make predictions
        print("Making predictions...")
        y_pred = xgb_regressor.predict(X_test_all).reshape(-1, 1)  # Reshape to match y_test_all format
        # Ensure no negative predictions
        y_pred = np.maximum(y_pred, 0)
        
        # Calculate metrics
        r2_score = xgb_regressor.score(X_test_all, y_test_all.ravel())
        mae_value = MAE(y_test_all, y_pred)
        mape_value = MAPE(y_test_all, y_pred)
        
        # Calculate q-score - vectorized version
        print("Calculating performance metrics...")
        
        # Vectorized q-score calculation
        y_true_flat = y_test_all.flatten()
        y_pred_flat = y_pred.flatten() if y_pred.ndim > 1 else y_pred
        
        # Find indices where both values are non-zero
        valid_indices = (y_true_flat != 0) & (y_pred_flat != 0)
        
        if np.any(valid_indices):
            ratios = np.maximum(
                y_pred_flat[valid_indices] / y_true_flat[valid_indices],
                y_true_flat[valid_indices] / y_pred_flat[valid_indices]
            )
            q_score_mean = np.mean(ratios)
        else:
            q_score_mean = 0
        
        # Time prediction performance (10 iterations)
        print("Measuring prediction performance...")
        total_duration = 0
        total_read = 0
        total_write = 0
        
        for _ in range(10):
            io_before = psutil.disk_io_counters()
            t3_start = process_time()
            preds = xgb_regressor.predict(X_test_all)
            preds = np.maximum(preds, 0)  # Ensure no negative predictions
            t3_stop = process_time()
            io_after = psutil.disk_io_counters()
            
            total_duration += (t3_stop - t3_start)
            total_read += io_after.read_count - io_before.read_count
            total_write += io_after.write_count - io_before.write_count
        
        avg_pred_time_microsec = (total_duration / 10) / len(y_pred) * 1000000
        avg_reads = total_read / 10 / len(y_pred)
        avg_writes = total_write / 10 / len(y_pred)
        
        # Save the model using joblib instead of pickle for better efficiency
        if SAVE_INTERMEDIATE_MODELS or sample_size == max_size:
            print("Saving model...")
            filename = f'../large_files/LearnedModels/contain/XGB/{dataset_name}_xgb_{sample_size}_{training_time:.2f}s_{mape_value:.2%}_{mae_value:.2f}.joblib'
            joblib.dump(xgb_regressor, filename, compress=3)
            # Get model file size in KB
            model_size_kb = os.path.getsize(filename) / 1024
            print(f"Model size: {model_size_kb:.2f} KB")
        else:
            model_size_kb = 0  # Set to 0 if model wasn't saved
        
        # Print results
        print(f"\nResults for {dataset_name}, Sample Size: {sample_size}")
        print(f"Grid Search Time: {grid_search_time:.2f}s, Training Time: {training_time:.2f}s")
        print(f"XGBoost Parameters: {best_params}")
        print(f"Performance: R² = {r2_score:.4f}, MAE = {mae_value:.2f}, MAPE = {mape_value:.2%}")
        print(f"q-score: {q_score_mean:.2f}")
        print(f"Prediction time: {avg_pred_time_microsec:.4f} μs/sample")
        print(f"I/O: Reads={avg_reads:.6f}, Writes={avg_writes:.6f}")
        print("-" * 80)
        
        # Plot actual vs predicted only for the maximum scale
        if sample_size == adjusted_scales[-1]:  # Check if this is the maximum scale
            print("Generating prediction scatter plot...")
            plt.figure(figsize=(10, 8))
            plt.scatter(y_test_all, y_pred, s=0.5, alpha=0.5)
            plt.xlabel('True Values')
            plt.ylabel('Predictions')
            plt.title(f"{dataset_name} - Sample Size: {sample_size} (Maximum)")
            plt.grid(True, alpha=0.3)
            
            # Add diagonal line for perfect predictions
            max_val = max(np.max(y_test_all), np.max(y_pred))
            plt.plot([0, max_val], [0, max_val], 'r--', alpha=0.5)
            
            plt.savefig(f"../large_files/LearnedModels/contain/XGB/visualizations/{dataset_name}_{sample_size}_prediction.png", dpi=150)
            plt.close()  # Close to free memory
            
            # Plot feature importance
            print("Generating feature importance plot...")
            plt.figure(figsize=(10, 6))
            feature_names = ['X1', 'Y1', 'X2', 'Y2', 'Rectangle Density']
            xgb.plot_importance(xgb_regressor, max_num_features=5)
            plt.title(f"Feature Importance - {dataset_name}")
            plt.tight_layout()
            plt.savefig(f"../large_files/LearnedModels/contain/XGB/visualizations/{dataset_name}_feature_importance.png", dpi=150)
            plt.close()
            
            # Create a scatter plot comparing predicted vs real values for first 100 rectangles
            print("Generating side-by-side comparison plot...")
            
            # Get predictions for first 100 test samples
            sample_indices = range(min(100, len(X_test_all)))
            X_sample = X_test_all[sample_indices]
            y_sample_true = y_test_all[sample_indices].flatten()
            y_sample_pred = xgb_regressor.predict(X_sample)
            # Ensure no negative predictions
            y_sample_pred = np.maximum(y_sample_pred, 0)
            
            plt.figure(figsize=(20, 10))
            plt.scatter(range(len(sample_indices)), y_sample_pred, c='green', 
                        label='Predicted number of objects (XGBoost)', alpha=0.7, s=100)
            plt.scatter(range(len(sample_indices)), y_sample_true, c='blue', 
                        label='Real number of objects', alpha=0.7, s=100)
            
            plt.title(f'{dataset_name} - First {len(sample_indices)} Rectangles: Predicted vs Real Values', fontsize=16)
            plt.xlabel('Rectangle Index', fontsize=14)
            plt.ylabel('Number of objects in rectangle', fontsize=14)
            plt.legend(fontsize=12)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            
            # Save the plot
            plt.savefig(f"../large_files/LearnedModels/contain/XGB/visualizations/{dataset_name}_comparison_plot.png", dpi=150)
            plt.close()
        
        # Store results in list (more efficient than DataFrame concat)
        result_row = {
            'Dataset': dataset_name,
            'Sample_Size': sample_size,
            'Training_Time': training_time,
            'Best_Params': str(best_params),
            'R2_Score': r2_score,
            'MAE': mae_value,
            'MAPE': float(mape_value),
            'Q_Score': q_score_mean,
            'Pred_Time_Microseconds': avg_pred_time_microsec,
            'IO_Reads': avg_reads,
            'IO_Writes': avg_writes,
            'Model_Size_KB': model_size_kb,
            'Max_Count': max_count,
            'Min_Count': min_count,
            'Mean_Count': mean_count,
            'Median_Count': median_count,
            'Total_Samples': total_samples
        }
        
        dataset_results_list.append(result_row)
        all_results_list.append(result_row)
        
        # Clean up to free memory
        if sample_size != max_size:  # Don't delete for max size as we might need it
            del X_train_sample, y_train_sample, xgb_regressor
            gc.collect()
    
    # Save results for this dataset
    print(f"Saving results for {dataset_name}...")
    dataset_results = pd.DataFrame(dataset_results_list)
    dataset_results.to_csv(f'../large_files/LearnedModels/contain/XGB/results/{dataset_name}_results.csv', index=False)
    
    # Clear memory before next dataset
    del X_train, X_test_all, y_train, y_test_all, Rectangles, Y, rectanglesDensity
    gc.collect()
    
# Save all results
print("Saving combined results...")
all_results = pd.DataFrame(all_results_list)
all_results.to_csv('../large_files/LearnedModels/contain/XGB/all_results.csv', index=False)

print("All processing completed and results saved.")
monitor_memory()

Using 29 of 30 available CPU cores
Loading spatial statistics...
Finding dataset files...
Found 14 datasets to process


Processing datasets:   0%|          | 0/14 [00:00<?, ?it/s]

Memory usage: 242.00 MB

Processing dataset: craftwaysorted
Universe boundaries for craftwaysorted: (-175.2000514, -65.2458821, 175.3397782, 69.6673353)
Loading data from ../large_files/resultsContains/craftwaysorted_results.csv
Parsing MBR coordinates...



Basic statistics for craftwaysorted dataset:
Max count: 108929.0
Min count: 0.0
Mean count: 1705.23
Median count: 0.00
Total samples: 21822

Calculating rectangle densities...
Splitting data into train and test sets...
Training set size: 17457

Training with sample size: 17457
Memory usage: 246.52 MB
Performing grid search for optimal parameters...
Fitting 3 folds for each of 27 candidates, totalling 81 fits


Grid search complete in 256.51s
Best parameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}
Training XGBoost model...


...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>